## 환경설정


API KEY 를 설정합니다.


In [1]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

True

In [2]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

아래는 기본적인 RAG 구조 이해를 위한 뼈대코드(skeleton code) 입니다.

각 단계별 모듈의 내용을 앞으로 상황에 맞게 변경하면서 문서에 적합한 구조를 찾아갈 수 있습니다.

(각 단계별로 다양한 옵션을 설정하거나 새로운 기법을 적용할 수 있습니다.)

In [4]:
# 단계 1: 문서 로드(Load Documents)
loader = PyMuPDFLoader("토양수분, 칼슘부족 및 주야간 습도에 따른 고랭지배추 딥번 발생.pdf")
docs = loader.load()
print(f"문서의 페이지수: {len(docs)}")

문서의 페이지수: 12


페이지의 내용을 출력합니다.

In [5]:
print(docs[1].page_content)

Horticultural Science and Technology
739
토양수분, 칼슘부족 및 주야간 습도에 따른 고랭지배추 팁번 발생
치명적인 영향을 미친다. 특히 고랭지배추는 결구된 형태로 수확되어 유통되기 때문에 포기 속에 발생한 팁번(tipburn)은 절개
하지 않으면 알 수 없어서 구입 후 식가공 과정에서 발견될 경우 고랭지배추에 대한 좋지 않은 이미지를 줄 수도 있다. 최근 한여
름에 재배되는 고랭지배추에 발생되고 있는 팁번은 배추의 품질 저하는 물론 수량 감소로 이어지고 있다. 특히 여름철 고온건
조기에 발생이 잘되는 팁번은 그 후 고온 다습을 맞아 2차적으로 무름 증상이 발생하면 일명 ‘꿀통 배추’가 되어 전혀 수확할 
수 없게 된다. 팁번은 해에 따라 간헐적으로 나타나기 때문에 고랭지배추 재배자들은 아직 완전한 극복 방법을 찾지 못하고 있
다. 팁번은 칼슘 부족에 의한 생리 장해로 100여 년 전에 밝혀졌으나 여러 가지 원인이 복합적으로 작용하여 아직도 완벽하게 
해결하지 못하고 있다(Saure, 1998). 팁번은 세포 내에 칼슘이 부족한 조건에서 polygalacturonase의 작용으로 pectic 
polysaccharide가 붕괴되고 세포벽이 와해되어 세포의 괴사가 일어나는 생리 장해(Selling et al., 2000)로, 토양이나 식물체의 
절대적인 칼슘 부족에 의한 경우보다는 과습, 건조(Balvoll, 1995), 고온 등 여러 가지 복잡한 환경조건의 변화에 의한 칼슘의 
흡수 장해 또는 부위 간 이동 분배의 불균형과 이를 더 부채질하는 생장 속도(Saure, 1998)도 관여하는 것으로 알려져 있다.
한편 칼슘의 흡수는 증산류를 따라 수동적으로 흡수·이동되므로(Clarkson, 1984) 주로 증발산이 많은 부위로 이동하게 되
어 칼슘을 많이 필요로 하는 신초 부위로 이동이 적다(Cho et al., 1994). 그러나 증산류에만 의존하는 것은 아니고 근압에 의해
서도 좌우되므로(Engels, 1999) 근권부 환경도 영향

`metadata` 를 확인합니다.

In [12]:
docs[2].__dict__

{'id': None,
 'metadata': {'producer': 'Acrobat Distiller 9.0.0 (Windows)',
  'creator': 'PScript5.dll Version 5.2.2',
  'creationdate': '2021-12-08T17:28:25+09:00',
  'source': 'data/토양수분, 칼슘부족 및 주야간 습도에 따른 고랭지배추 딥번 발생.pdf',
  'file_path': 'data/토양수분, 칼슘부족 및 주야간 습도에 따른 고랭지배추 딥번 발생.pdf',
  'total_pages': 12,
  'format': 'PDF 1.4',
  'title': '',
  'author': '',
  'subject': '',
  'keywords': '',
  'moddate': '2021-12-08T17:40:08+09:00',
  'trapped': '',
  'modDate': "D:20211208174008+09'00'",
  'creationDate': "D:20211208172825+09'00'",
  'page': 2},
 'page_content': "740\nHorticultural Science and Technology\n토양수분, 칼슘부족 및 주야간 습도에 따른 고랭지배추 팁번 발생\nScientific, USA)으로 분석하였다.\n \n주야간 습도 조절에 따른 배추 내외부 온습도 및 포차의 변화 특성\n배추 포기 내부의 온습도의 변화를 알아보기 위해 배추묘(‘수호’, 농우바이오)를 직경 36cm의 폿트에 정식하여 생육 상자\n(W × D × H, 1 × 1 × 1m)에서 재배하면서 제습기(DHJ-067BNB, Winix)와 가습기(SGM-L124, Geek)를 활용하여 주간제습\n/야간제습 및 주간제습/야간가습 처리하고 배추 포기 내외의 온습도 및 포차를 조사하였다. 배추 포기 내외부의 온습도는 온도\n센서(Almemo Thermo Za 9020-Fs, Arborn, Germany)와 습도

In [8]:
# 단계 2: 문서 분할(Split Documents)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=700, chunk_overlap=50)
split_documents = text_splitter.split_documents(docs)
print(f"분할된 청크의수: {len(split_documents)}")

분할된 청크의수: 46


In [9]:
print(split_documents[1].page_content)

https://doi.org/10.7235/HORT.20210065
토양수분, 칼슘부족 및 주야간 습도에 따른 고랭지배추 
팁번 발생
김기덕*
 · 서종택 · 김창석 · 지삼녀
국립식량과학원 고령지농업연구소
Tipburn Occurrence According to Soil Moisture, 
Calcium Deficiency, and Day/Night Relative 
Humidity Regime in Highland Kimchi Cabbage
Ki-Deog Kim*
, Jong-Taek Suh, Chang-Seok Kim, and Sam-Nyu Jee
Highland Agriculture Research Institute, National Institute of Crop Science, RDA, Pyeongchang 25342, 
Korea
*Corresponding author: kkd1414@korea.kr
Abstract
Tipburn occurs occasionally in Kimchi cabbage, and lowers quality and yield. This study was 
carried out to understand the causes and reduce the occurrence of tipburn. Daily calcium uptake


In [11]:
import tiktoken
from langchain.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# 단계 1: PDF 문서 로드
loader = PyMuPDFLoader("토양수분, 칼슘부족 및 주야간 습도에 따른 고랭지배추 딥번 발생.pdf")
docs = loader.load()
print(f"문서의 페이지 수: {len(docs)}")

# 단계 2: 문서 분할
text_splitter = RecursiveCharacterTextSplitter(chunk_size=700, chunk_overlap=50)
split_documents = text_splitter.split_documents(docs)
print(f"분할된 청크 수: {len(split_documents)}")

# 단계 3: 토큰 수 및 예상 비용 계산
def estimate_gpt4o_mini_cost(texts, model="gpt-4o-mini"):
    """GPT-4o-mini 모델을 사용할 때 예상 토큰 수 및 비용 계산"""
    encoding = tiktoken.encoding_for_model(model)
    total_tokens = sum(len(encoding.encode(text.page_content)) for text in texts)

    # GPT-4o-mini 비용 (예상 값, OpenAI 공식 문서에서 최신 정보 확인 필요)
    cost_per_1k_input = 0.0005  # 입력 토큰당 $0.0005 (예제 값)

    estimated_cost = (total_tokens / 1000) * cost_per_1k_input
    return total_tokens, estimated_cost

tokens, cost = estimate_gpt4o_mini_cost(split_documents)

print(f"예상 총 토큰 개수: {tokens}")
print(f"예상 비용: ${cost:.6f}")  # 소수점 6자리까지 출력


문서의 페이지 수: 12
분할된 청크 수: 46
예상 총 토큰 개수: 12072
예상 비용: $0.006036


In [2]:
# 단계 3: 임베딩(Embedding) 생성
embeddings = OpenAIEmbeddings() #돈듭니다.
embeddings

NameError: name 'OpenAIEmbeddings' is not defined

In [7]:
# 단계 4: DB 생성(Create DB) 및 저장
# 벡터스토어를 생성합니다.
vectorstore = FAISS.from_documents(documents=split_documents, embedding=embeddings) #FAISS 는 DB

NameError: name 'split_documents' is not defined

In [27]:
# 1. FAISS 벡터 DB 저장 코드
vectorstore.save_local("faiss_index") #

In [19]:
import faiss

index = faiss.read_index("faiss_index/index.faiss")  
print("FAISS 벡터 차원 수:", index.d)

FAISS 벡터 차원 수: 1536


In [29]:
from langchain.vectorstores import FAISS
from langchain.embeddings import FakeEmbeddings

# FakeEmbeddings로 임베딩 함수 제공 (차원 수에 맞게 설정)
fake_embeddings = FakeEmbeddings(size=1536)

# FAISS 로드
vectorstore = FAISS.load_local("faiss_index", embeddings=fake_embeddings, allow_dangerous_deserialization=True)

# 검색 테스트
for doc in vectorstore.similarity_search("gd"):
    print(doc.page_content)


Fertilizer based on soil testing
Lack of calcium 
content in soil
Fertilization
Natural supply
+++
－
+++
－
Control of fertilizer
-
z+++ Enough, ++ Well, + Good, －Not good or insufficient.
호’, 농우바이오)를 재배하다가 칼슘결제처리일 오전 10시에 뿌리를 세척하고 칼슘을 제거한 양액에서 2, 4, 8, 16, 24 및 48시
간 동안 결제처리하였다. 이 후 표준 양액에서 40일간 더 재배한 후 팁번 발생을 조사하였다(Table 1). 칼슘결제 시 양액 조제는 
양이온교환수지로 칼슘을 제거시킨 물을 사용하였다. 재배 종료 후 결제처리기간 이외의 재배기간 중 1주일 간격으로 양액을 
갱신하였다. 또한 0.3%의 염화 칼슘 용액을 결구기에 5일 간격으로 3회 살포하였다. 팁번이 발생한 잎과 발생하지 않은 잎의 
잎 중앙 부위와 끝부분, 그리고 포기 내외부 잎의 칼슘 함량은 전자현미경 장착 EDS system(FE-SEM+EDS, Thermo fisher
Horticultural Science and Technology
741
토양수분, 칼슘부족 및 주야간 습도에 따른 고랭지배추 팁번 발생
결과 및 고찰
배추의 칼슘 결제처리 및 칼슘액 엽면살포에 따른 팁번 발생 
칼슘 결제에 따른 배추의 칼슘 결핍 발생 양상을 알아보기 위하여 칼슘을 제거한 양액에서 일정기간 처리한 후 배추의 팁번 
발생을 조사하였다(Table 1). 팁번은 24시간, 48시간 칼슘 결제처리한 구에서 모두 나타났다. 칼슘 엽면 살포는 칼슘 결핍증 발
생을 줄이는 효과는 보이지 않았으나 결핍 정도는 다소 낮추는 효과는 있었다. 결구 시작기의 칼슘 결제처리 시 팁번 나타내는 
A
B
C
D
Fig. 1. Daily changes of relative humidity by treatment in a pilot-scale greenhouse controlled 

In [30]:

# FAISS 벡터 DB 불러오기 코드
from langchain.vectorstores import FAISS
from langchain.embeddings import FakeEmbeddings

# FAISS 저장할 때 사용한 벡터 차원 수와 동일하게 설정 (예: 1536)
fake_embeddings = FakeEmbeddings(size=1536)
vectorstore = FAISS.load_local("faiss_index", embeddings=fake_embeddings, allow_dangerous_deserialization=True)

# 올바른 로드 방법: FakeEmbeddings 인스턴스를 넘겨야 함
#vectorstore = FAISS.load_local("faiss_index", fake_embeddings)

In [ ]:
for doc in vectorstore.similarity_search("구글"):
    print(doc.page_content)

In [31]:
# 단계 5: 검색기(Retriever) 생성
# 문서에 포함되어 있는 정보를 검색하고 생성합니다.
retriever = vectorstore.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'FakeEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001C773A40510>, search_kwargs={})

In [32]:
retriever.invoke("ㅁㄴㅇㄻㄴㅇㄹㄷㅈㄱㅈ")

[Document(id='070024cc-4280-49ab-87fd-b87af416eb7d', metadata={'producer': 'Acrobat Distiller 9.0.0 (Windows)', 'creator': 'PScript5.dll Version 5.2.2', 'creationdate': '2021-12-08T17:28:25+09:00', 'source': 'data/토양수분, 칼슘부족 및 주야간 습도에 따른 고랭지배추 딥번 발생.pdf', 'file_path': 'data/토양수분, 칼슘부족 및 주야간 습도에 따른 고랭지배추 딥번 발생.pdf', 'total_pages': 12, 'format': 'PDF 1.4', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'moddate': '2021-12-08T17:40:08+09:00', 'trapped': '', 'modDate': "D:20211208174008+09'00'", 'creationDate': "D:20211208172825+09'00'", 'page': 7}, page_content='인 8월 30일경에 야간 동안의 습도가 낮고 토양 수분포텐셜이 낮은 조건은 구 내부 신초엽에 팁번이 발생할 수 있는 필요조건을 \nTable 3. Tipburn occurrence by leaf order according to soil moisture conditions by growth stage and air humidity control at \nnight\nSoil water conditionz\nAir humidity control \nDays after planting\nControl\nHumidified \nDehumidified\nEarly \n(1－20)\nMiddle\n(21－42)\nLate\n(43－63)\nLeaf order (Outer 1↔ Inner 70)\n◯\n◯\n◯\nNOy\nNO\nNO\n+\n◯\n◯\nNO\

검색기에 쿼리를 날려 검색된 chunk 결과를 확인합니다.

In [33]:
# 단계 6: 프롬프트 생성(Create Prompt)
# 프롬프트를 생성합니다.
prompt = PromptTemplate.from_template(
    """당신은 질문에 답하는 업무를 수행하는 도우미입니다.
다음에 제공된 문맥을 사용하여 질문에 답하세요.
만약 답을 모른다면, 모른다고 솔직하게 말하세요.
답변은 한글로 작성하세요.

#Context: 
{context}

#Question:
{question}

#Answer:"""
)

In [34]:
# 단계 7: 언어모델(LLM) 생성
# 모델(LLM) 을 생성합니다.
llm = ChatOpenAI(model_name="gpt-4o", temperature=0)

In [35]:
# 단계 8: 체인(Chain) 생성
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

생성된 체인에 쿼리(질문)을 입력하고 실행합니다.

In [37]:
# 체인 실행(Run Chain)
# 문서에 대한 질의를 입력하고, 답변을 출력합니다.
question = "수경재배에대해"
response = chain.invoke(question)
print(response)

문맥에서 수경재배에 대한 정보는 제공되지 않았습니다. 따라서 수경재배에 대해 답변할 수 없습니다.


## 전체 코드

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

# 단계 1: 문서 로드(Load Documents)
loader = PyMuPDFLoader("data/SPRI_AI_Brief_2023년12월호_F.pdf")
docs = loader.load()

# 단계 2: 문서 분할(Split Documents)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
split_documents = text_splitter.split_documents(docs)

# 단계 3: 임베딩(Embedding) 생성
embeddings = OpenAIEmbeddings()

# 단계 4: DB 생성(Create DB) 및 저장
# 벡터스토어를 생성합니다.
vectorstore = FAISS.from_documents(documents=split_documents, embedding=embeddings)

# 단계 5: 검색기(Retriever) 생성
# 문서에 포함되어 있는 정보를 검색하고 생성합니다.
retriever = vectorstore.as_retriever()

# 단계 6: 프롬프트 생성(Create Prompt)
# 프롬프트를 생성합니다.
prompt = PromptTemplate.from_template(
    """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the question. 
If you don't know the answer, just say that you don't know. 
Answer in Korean.

#Context: 
{context}

#Question:
{question}

#Answer:"""
)

# 단계 7: 언어모델(LLM) 생성
# 모델(LLM) 을 생성합니다.
llm = ChatOpenAI(model_name="gpt-4o", temperature=0)

# 단계 8: 체인(Chain) 생성
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
# 체인 실행(Run Chain)
# 문서에 대한 질의를 입력하고, 답변을 출력합니다.
question = "삼성전자가 자체 개발한 AI 의 이름은?"
response = chain.invoke(question)
print(response)